In [1]:
pip install googlemaps


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
pip install openpyxl


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import time
import googlemaps
import pandas as pd
import csv
from pprint import pprint
import openpyxl

In [3]:
API_KEY = open('API.txt','r').read()
map_client = googlemaps.Client(API_KEY)

Getting the cordinates of the all neighborhoods in the property dataset

In [ ]:
# List of neighborhood names in Baltimore
neighborhood_names = [
    "Mount Washington",
    "Fells Point",
    "Clifton",
    "Downtown",
    "Arlington",
    "Carroll",
    "Purnell",
    "Greater Northwood Covenant Association",
    "Northwood",
    "Locust Point",
    "Lakeland",
    "Bolton Hill",
    "Highlandtown",
    "Gwynn Oak",
    "Druid Hill Park",
    "Otterbein",
    "Franklintown",
    "Waverly",
    "Charles Village",
    "Old Goucher",
    "Cherry Hill",
    "South Baltimore",
    "Woodberry",
    "Govans",
    "Sharp Leadenhall",
    "Tuscany - Canterbury",
    "East Case",
    "Grove Park",
    "Mid-Town Belvedere",
    "New North Roland Park - Poplar Hill",
    "Mosher",
    "Middle East",
    "Riverside",
    "Garwyn Oaks",
    "Mount Holly",
    "Elwood Park",
    "Federal Hill-Montgomery",
    "West Forest Park",
    "Inner Harbor",
    "Ridgely's Delight",
    "Ednor Gardens - Lakeside",
    "Brooklyn",
    "Raspeburg",
    "Medfield",
    "Panway - Braddish",
    "Forest Park",
    "Harford - Echodale - Perring Parkway",
    "Station North",
    "Upper Fells Point",
    "Hampden",
    "Pen Lucy",
    "Oakenshawe",
    "Harwood",
    "Coldstream - Homestead - Montebello",
    "Bridgeview-Greenlawn",
    "Gay Street",
    "Franklin",
    "Pigtown",
    "Orangeville",
    "Better Waverly",
    "Butchers Hill",
    "Evergreen Lawn",
    "Morrell Park",
    "Glenham-Belford",
    "Western",
    "Madison - Eastend",
    "Walbrook",
    "Abell",
    "East Baltimore Midway",
    "Remington",
    "Coppin Heights",
    "Hollins Market",
    "Lauraville",
    "Mondawmin",
    "Canton",
    "Barre Circle",
    "Barclay",
    "Windsor Hills",
    "Wyndhurst",
    "Wyman Park",
    "Rosemont",
    "NW Community Action",
    "Winchester",
    "Woodring",
    "Curtis Bay",
    "Milton - Montford",
]

# Create an empty list to store neighborhood data
neighborhood_data = []

# Function to geocode neighborhood names and get coordinates and radii
def geocode_neighborhoods(names):
    for name in names:
        try:
            # Geocode the neighborhood name
            geocode_result = map_client.geocode(name + ', Baltimore, MD, USA')

            # Extracting latitude and longitude from the geocoding result
            if geocode_result and 'geometry' in geocode_result[0]:
                location = geocode_result[0]['geometry']['location']
                latitude = location['lat']
                longitude = location['lng']

                radius = 2000

                # Create a dictionary with neighborhood data
                neighborhood_info = {
                    "name": name,
                    "location": f"{latitude},{longitude}",
                    "radius": radius
                }

                neighborhood_data.append(neighborhood_info)
            else:
                print(f"Could not geocode {name}")

        except Exception as e:
            print(f"Error geocoding {name}: {str(e)}")

# Calling the function to geocode neighborhood names
geocode_neighborhoods(neighborhood_names)

# Creating a DataFrame from the neighborhood data
neighborhood_df = pd.DataFrame(neighborhood_data)

# Print the DataFrame or save it to a CSV file
print(neighborhood_df)
neighborhood_df.to_csv('neighborhood_data.csv', index=False)

Pulling allowed gyms data for Baltimore

In [ ]:
# Load the neighborhood details from a CSV file into a DataFrame
neighborhood_df = pd.read_csv('neighborhood_data.csv')

# Function to fetch gyms in a neighborhood
def fetch_gyms(neighborhood):
    allowed_columns = ["place_id", "geometry"]  # Specifying the allowed columns

    gym_list = []
    response = map_client.places_nearby(
        location=neighborhood["location"],
        radius=neighborhood["radius"],
        type='gym',  
    )

    while 'results' in response:
        for result in response['results']:
            # Filter columns to keep only allowed ones
            filtered_result = {key: result[key] for key in allowed_columns}

            # Add neighborhood information
            filtered_result['neighborhood'] = neighborhood["location"]

            gym_list.append(filtered_result)

        time.sleep(2)  # Pause to avoid rate limiting issues
        if 'next_page_token' in response:
            response = map_client.places_nearby(
                location=neighborhood["location"],
                radius=neighborhood["radius"],
                type='gym',
                page_token=response['next_page_token']
            )
        else:
            break

    return gym_list

# Fetch gyms for each neighborhood in the DataFrame
all_gyms = []

for index, row in neighborhood_df.iterrows():
    neighborhood_info = {
        "name": row["name"],
        "location": row["location"],
        "radius": row["radius"]
    }
    neighborhood_gyms = fetch_gyms(neighborhood_info)
    all_gyms.extend(neighborhood_gyms)

# Create a DataFrame from the combined list of gyms
gym_df = pd.DataFrame(all_gyms)

# Save the merged dataset to an Excel file
gym_df.to_excel('Allowed_gyms.xlsx', index=False)

Pulling allowed restaurant data for Baltimore

In [7]:
# Load the neighborhood details from a CSV file into a DataFrame
neighborhood_df = pd.read_csv('neighborhood_data.csv')

# Function to fetch gyms in a neighborhood
def fetch_business(neighborhood):
    allowed_columns = ["place_id", "geometry"]  # Specifying the allowed columns

    gym_list = []
    response = map_client.places_nearby(
        location=neighborhood["location"],
        radius=neighborhood["radius"],
        type='restaurant',  
    )

    while 'results' in response:
        for result in response['results']:
            # Filter columns to keep only allowed ones
            filtered_result = {key: result[key] for key in allowed_columns}

            # Add neighborhood information
            filtered_result['neighborhood'] = neighborhood["location"]

            gym_list.append(filtered_result)

        time.sleep(2)  # Pause to avoid rate limiting issues
        if 'next_page_token' in response:
            response = map_client.places_nearby(
                location=neighborhood["location"],
                radius=neighborhood["radius"],
                type='restaurant',
                page_token=response['next_page_token']
            )
        else:
            break

    return gym_list

# Fetch gyms for each neighborhood in the DataFrame
all_gyms = []

for index, row in neighborhood_df.iterrows():
    neighborhood_info = {
        "name": row["name"],
        "location": row["location"],
        "radius": row["radius"]
    }
    neighborhood_gyms = fetch_business(neighborhood_info)
    all_gyms.extend(neighborhood_gyms)

# Create a DataFrame from the combined list of gyms
gym_df = pd.DataFrame(all_gyms)

# Save the merged dataset to an Excel file
gym_df.to_excel('Allowed_restaurant.xlsx', index=False)

Pulling allowed supermarkets data for Baltimore

In [5]:
# Load the neighborhood details from a CSV file into a DataFrame
neighborhood_df = pd.read_csv('neighborhood_data.csv')

# Function to fetch supermarkets in a neighborhood 
def fetch_supermarket(neighborhood):
    allowed_columns = ["place_id", "geometry"]  # Specifying the allowed columns

    supermarket_list = []
    response = map_client.places_nearby(
        location=neighborhood["location"],
        radius='1500',
        type='supermarket',  
    )

    while 'results' in response:
        for result in response['results']:
            # Filter columns to keep only allowed ones
            filtered_result = {key: result[key] for key in allowed_columns}

            # Add neighborhood information
            filtered_result['neighborhood'] = neighborhood["location"]

            supermarket_list.append(filtered_result)

        time.sleep(2)  # Pause to avoid rate limiting issues
        if 'next_page_token' in response:
            response = map_client.places_nearby(
                location=neighborhood["location"],
                radius='1500',
                type='supermarket',
                page_token=response['next_page_token']
            )
        else:
            break

    return supermarket_list

# Fetch supermarkets for each neighborhood in the DataFrame
all_supermarket = []

for index, row in neighborhood_df.iterrows():
    neighborhood_info = {
        "name": row["name"],
        "location": row["location"],
        "radius": row["radius"]
    }
    neighborhood_supermarket = fetch_supermarket(neighborhood_info)
    all_supermarket.extend(neighborhood_supermarket)

# Create a DataFrame from the combined list of supermarkets
supermarket_df = pd.DataFrame(all_supermarket)

# Save the merged dataset to an Excel file
supermarket_df.to_excel('Allowed_supermarkets.xlsx', index=False)